In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Use pygeos in geopandas
os.environ['USE_PYGEOS'] = '0'

import glob
import warnings
import h5py as h5
import ruamel.yaml as yaml

from tqdm.auto import tqdm
import multiresticodm.spatial_interaction_model as SIM_Model

from multiresticodm.utils import *
from multiresticodm.notebook_functions import *


warnings.simplefilter("ignore")
# mpl.rcParams['agg.path.chunksize'] = 10000

In [3]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

## Import outputs

In [4]:
# Expertiment id
geometry_name = 'lsoas_to_msoas'
origin_geometry_name = 'lsoa'
destination_geometry_name = 'msoa'

model_name = f"HarrisWilson"
dataset_name = f'Cambridge_dataset'
output_folders = ['230517-155009_Cambridge_dataset','230517-155253_Cambridge_dataset', '230517-155328_Cambridge_dataset', '230517-155111_Cambridge_dataset']

In [5]:
low_noise_data = {"TotallyConstrained":{},"ProductionConstrained":{}}
high_noise_data = {"TotallyConstrained":{},"ProductionConstrained":{}}
learned_noise_data = {"TotallyConstrained":{},"ProductionConstrained":{}}

low_noise_config = {"TotallyConstrained":{},"ProductionConstrained":{}}
high_noise_config = {"TotallyConstrained":{},"ProductionConstrained":{}}
learned_noise_config = {"TotallyConstrained":{},"ProductionConstrained":{}}

keys_of_interest = ["alpha","beta","loss","pred_dest_sizes"]

for output_folder in output_folders:
    # Define directory
    outputs_data_path = f'../../../temp_backup/HarrisWilsonbackup/{output_folder}/data/'
    config_path = f"../../../temp_backup/HarrisWilsonbackup/{output_folder}/config/run_cfg.yml"
    for filepath in tqdm(sorted(glob.glob(os.path.join(outputs_data_path,"*")))):
        # Read config
        with open(os.path.join(filepath,'config.yml'), "r") as stream:
            config = yaml.safe_load(stream)
        data_slice = read_h5_data(filepath,keys=keys_of_interest)
        sigma = config['HarrisWilson']['Training']['true_parameters'].get('sigma',-1)
        sim_type = config['HarrisWilson']['FlowModel'].replace('ABM','')
        if sim_type == 'TotalConstrained':
            sim_type = 'TotallyConstrained'
        if sigma <= 0.05 and sigma >= 0:
            low_noise_config[sim_type] = config
            if not low_noise_config[sim_type]['seed'] in low_noise_data[sim_type]: 
                low_noise_data[sim_type][low_noise_config[sim_type]['seed']] = dict(zip(keys_of_interest,[None]*len(keys_of_interest)))
            for k in keys_of_interest:
                if low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k] is None:
                    low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k] = data_slice[k]
                else:
                    low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k] = np.append(
                        low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k],
                        data_slice[k],
                        axis=0
                    )
        elif sigma > 0.05:
            high_noise_config[sim_type] = config
            if not high_noise_config[sim_type]['seed'] in high_noise_data[sim_type]: 
                high_noise_data[sim_type][high_noise_config[sim_type]['seed']] = dict(zip(keys_of_interest,[None]*len(keys_of_interest)))
            for k in keys_of_interest:
                if high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k] is None:
                    high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k] = data_slice[k]
                else:
                    high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k] = np.append(
                        high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k],
                        data_slice[k],
                        axis=0
                    )
        else:
            learned_noise_config[sim_type] = config
            if not learned_noise_config[sim_type]['seed'] in learned_noise_data[sim_type]: 
                learned_noise_data[sim_type][learned_noise_config[sim_type]['seed']] = dict(zip(keys_of_interest,[None]*len(keys_of_interest)))
            for k in keys_of_interest:
                if learned_noise_data[sim_type][learned_noise_config[sim_type]['seed']][k] is None:
                    learned_noise_data[sim_type][learned_noise_config[sim_type]['seed']][k] = data_slice[k]
                else:
                    learned_noise_data[sim_type][learned_noise_config[sim_type]['seed']][k] = np.append(
                        learned_noise_data[sim_type][learned_noise_config[sim_type]['seed']][k],
                        data_slice[k],
                        axis=0
                    )

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [6]:
BAREBONE_CONFIG = {
  "log_level": "info",
  "sweep_mode": True,
  "inputs": {
    "n_workers": 6,
    "n_threads": 7,
    "device": "cpu",
    "in_directory": "./data/inputs/",
    "dataset": "cambridge_work_commuter_lsoas_to_msoas",
    "load_experiment": "",
    "seed": {
      "sweep": {
        "default": 0,
        "range": [
          "0:99:1"
        ]
      }
    },
    "to_learn": {
      "sweep": {
        "default": [
          "alpha",
          "beta"
        ],
        "range": [
          [
            "alpha",
            "beta"
          ],
          [
            "alpha",
            "beta"
          ],
          [
            "alpha",
            "beta",
            "sigma"
          ]
        ],
        "coupled": True,
        "target_name": "sigma"
      }
    },
    "data": {
      "origin_demand": {
        "file": "origin_demand_sum_normalised.txt"
      },
      "destination_attraction_ts": {
        "file": "destination_attraction_time_series_sum_normalised.txt"
      },
      "cost_matrix": {
        "file": "cost_matrices/clustered_facilities_sample_20x20_20_01_2023_sample_20x20_clustered_facilities_ripleys_k_500_euclidean_points%_prob_origin_destination_adjusted_normalised_boundary_only_edge_corrected_cost_matrix_sum_normalised.txt"
      },
      "ground_truth_table": {
        "file": "table_lsoas_to_msoas.txt"
      },
      "total_cost_by_origin": {
        "file": "lsoas_total_distance_to_work.txt"
      }
    }
  },
  "spatial_interaction_model": {
    "name": {
      "sweep": {
        "default": "TotallyConstrained",
        "range": [
          "TotallyConstrained",
          "ProductionConstrained"
        ],
        "coupled": True,
        "target_name": "title",
      }
    },
    "grand_total": 33704,
    "parameters": {
      "bmax": 1.0,
      "alpha": 1.0,
      "beta": 1.0
    }
  },
  "harris_wilson_model": {
    "dt": 0.001,
    "parameters": {
      "noise_percentage": 0.0001,
      "epsilon": 1.0,
      "delta": 0.0,
      "sigma": {
        "sweep": {
          "default": 0.0141421356,
          "range": [
            0.0141421356,
            0.1414213562,
            None
          ]
        }
      }
    }
  },
  "training": {
    "num_steps": 1,
    "batch_size": 1,
    "N": 1000
  },
  "neural_network": {
    "disable_tqdm": True,
    "loss": {
      "loss_name": [
        "dest_attraction_ts_loss"
      ],
      "loss_function": [
        "mseloss"
      ],
      "loss_kwarg_keys": [
        []
      ]
    },
    "hyperparameters": {
      "num_layers": 1,
      "optimizer": "Adam",
      "learning_rate": 0.002,
      "biases": {
        "default": [
          0.0,
          4.0
        ],
        "layer_specific": {}
      },
      "nodes_per_layer": {
        "default": 20,
        "layer_specific": {}
      },
      "activation_funcs": {
        "default": "linear",
        "layer_specific": {
          "1": "abs"
        }
      }
    }
  },
  "experiments": [
    {
      "type": "SIM_NN",
      "comment": "Spatial Interaction Model parameter learning using Neural Networks",
      "disable_tqdm": False,
      "export_samples": True,
      "export_metadata": True,
      "overwrite": True
    }
  ],
  "outputs": {
    "chunk_size": 20000,
    "write_start": 1,
    "write_every": 1,
    "out_directory": "./data/outputs/",
    "out_group": "exp1",
    "title": {
      "sweep": {
        "default": "_total_constrained",
        "range": [
          "_total_constrained",
          "_row_constrained"
        ]
      }
    }
  },
  "experiment_type": "SIM_NN",
  "load_data": False,
  "datetime": "16_05_2023_20_09_04"
}

In [7]:
experiment_id = 'SIM_NN_SweepedNoise_16_05_2023_20_09_04'
outputs_path = os.path.join(
    '../../data/outputs/cambridge_work_commuter_lsoas_to_msoas/',
    experiment_id
)
makedir(outputs_path)
write_json(
    BAREBONE_CONFIG,
    os.path.join(
        outputs_path,
        'config.json'
    )
)


title_map = {"TotallyConstrained":"_total_constrained","ProductionConstrained":"_row_constrained"}
to_learn_map = {"low":['alpha','beta'],"high":['alpha','beta'],"learned":['alpha','beta','sigma']}
indx = 0
for prefix,name in {"tc":"TotallyConstrained","pc":"ProductionConstrained"}.items():
    for sigma in [0.0141421356, 0.1414213562, None]:
        noise_regime = sigma_to_noise_regime(sigma)
        data = globals()[(noise_regime+'_noise_data')][name]
        for seed,datum in data.items():
            # print(indx,'/',600)
            indx += 1
            BAREBONE_CONFIG['inputs']['seed'] = seed
            BAREBONE_CONFIG['spatial_interaction_model']['name'] = name
            BAREBONE_CONFIG['harris_wilson_model']['parameters']['sigma'] = sigma
            BAREBONE_CONFIG['outputs']['title'] = title_map[name]
            BAREBONE_CONFIG['inputs']['to_learn'] = to_learn_map[noise_regime]
            BAREBONE_CONFIG['sweep_mode'] = False
            
            sweep_id = os.path.join(
                f"seed_{seed}",
                f"sigma_{noise_regime}",
                f"title_{title_map[name]}"
            )
            sweep_outputs_path = os.path.join(
                outputs_path,
                'samples',
                sweep_id
            )
            makedir(sweep_outputs_path)
            write_json(
                BAREBONE_CONFIG,
                os.path.join(
                    sweep_outputs_path,
                    'metadata.json'
                )
            )
            with open(os.path.join(sweep_outputs_path,'outputs.log'), 'w') as f:
                f.write('DONE')
            
            datum['log_destination_attraction'] = np.log(datum['pred_dest_sizes']).reshape(
                BAREBONE_CONFIG['training']['N'],1,datum['pred_dest_sizes'].shape[1]
            )
            datum['total_loss'] = deepcopy(datum['loss'])
            del datum['loss']

            sweep_params = {
                "seed": BAREBONE_CONFIG['inputs']['seed'],
                "name": BAREBONE_CONFIG['spatial_interaction_model']['name'],
                "sigma": BAREBONE_CONFIG['harris_wilson_model']['parameters']['sigma'],
                "title": BAREBONE_CONFIG['outputs']['title'],
                "to_learn": BAREBONE_CONFIG['inputs']['to_learn'],
                "iter": BAREBONE_CONFIG['training']['N'] #np.arange(1,BAREBONE_CONFIG['training']['N'],1),
            }

            h5file = h5.File(
                os.path.join(
                    sweep_outputs_path,
                    "data.h5"
                ), 
                mode='w'
            )


            # Store experiment id
            h5group = h5file.create_group(experiment_id)

            # Store sweep configurations as attributes
            h5group.attrs.create("sweep_params",list(sweep_params.keys()))
            h5group.attrs.create("sweep_values",['none' if val is None else str(val) for val in sweep_params.values()])

            for dname in ['alpha','beta','log_destination_attraction','total_loss']:
                h5group.create_dataset(
                    dname,
                    data = datum[dname]
                )

            h5file.close()


In [15]:
datum['log_destination_attraction'].shape

(1000, 1, 13)

In [ ]:
for prefix,name in {"tc":"TotalConstrained","pc":"ProductionConstrained"}.items():
    for noise_regime in ['low_noise','high_noise','learned_noise']:
        
        signs, thetas, log_destination_attractions = prepare_for_export(
            globals()[(noise_regime+'_data')][name],
            take_mean=False
        )
        exec(f"{prefix}_{noise_regime}_signs = signs")
        exec(f"{prefix}_{noise_regime}_thetas = thetas")
        exec(f"{prefix}_{noise_regime}_log_destination_attractions = log_destination_attractions")        

In [ ]:
tc_low_noise_thetas[:,0]

In [ ]:
print('Total constrained SIM')
print(tc_low_noise_signs.shape)
print(tc_low_noise_thetas.shape)
print(tc_low_noise_log_destination_attractions.shape)
print('\n')
print(tc_high_noise_signs.shape)
print(tc_high_noise_thetas.shape)
print(tc_high_noise_log_destination_attractions.shape)
print('\n')
print(tc_learned_noise_signs.shape)
print(tc_learned_noise_thetas.shape)
print(tc_learned_noise_log_destination_attractions.shape)
print('\n')
print('Production constrained SIM')
print(pc_low_noise_signs.shape)
print(pc_low_noise_thetas.shape)
print(pc_low_noise_log_destination_attractions.shape)
print('\n')
print(pc_high_noise_signs.shape)
print(pc_high_noise_thetas.shape)
print(pc_high_noise_log_destination_attractions.shape)
print('\n')
print(pc_learned_noise_signs.shape)
print(pc_learned_noise_thetas.shape)
print(pc_learned_noise_log_destination_attractions.shape)

In [ ]:
print('Total constrained SIM')
print('low noise',np.min(tc_low_noise_thetas,axis=0),np.max(tc_low_noise_thetas,axis=0))
print('high noise',np.min(tc_high_noise_thetas,axis=0),np.max(tc_high_noise_thetas,axis=0))
print('learned noise',np.min(tc_learned_noise_thetas,axis=0),np.max(tc_learned_noise_thetas,axis=0))
print('\n')
print('Production constrained SIM')
print('low noise',np.min(pc_low_noise_thetas,axis=0),np.max(pc_low_noise_thetas,axis=0))
print('high noise',np.min(pc_high_noise_thetas,axis=0),np.max(pc_high_noise_thetas,axis=0))
print('learned noise',np.min(pc_learned_noise_thetas,axis=0),np.max(pc_learned_noise_thetas,axis=0))
print('\n')

In [ ]:
low_noise_metadata = prepare_config_for_export(BAREBONE_CONFIG,low_noise_config)
high_noise_metadata = prepare_config_for_export(BAREBONE_CONFIG,high_noise_config)
learned_noise_metadata = prepare_config_for_export(BAREBONE_CONFIG,learned_noise_config)

# Intensity

In [ ]:
cost_matrix = np.loadtxt('../data/inputs/cambridge_work_commuter_lsoas_to_msoas/cost_matrices/clustered_facilities_sample_20x20_20_01_2023_sample_20x20_clustered_facilities_ripleys_k_500_euclidean_points%_prob_origin_destination_adjusted_normalised_boundary_only_edge_corrected_cost_matrix_sum_normalised.txt')
# cost_matrix = pd.read_csv('../../NeuralABM/data/HarrisWilson/Cambridge_data/exp_clustered_facilities_sample_20x20_20_01_2023_sample_20x20_clustered_facilities_ripleys_k_500_euclidean_points%_prob_origin_destination_adjusted_normalised_boundary_only_edge_corrected_cost_matrix_max_normalised.csv',index_col=0)
# cost_matrix = cost_matrix.values
origin_demand = np.loadtxt('../data/inputs/cambridge_work_commuter_lsoas_to_msoas/origin_demand_sum_normalised.txt')

employment = pd.read_csv(f'../data/raw/cambridge_commuter/employment_survey_msoa.csv',header=None)
employment.columns = [f'{destination_geometry_name}_id','number_of_jobs']
destination_attraction = employment['number_of_jobs'].values.astype('float32')
destination_attraction /= np.sum(destination_attraction) # 10000

table = np.loadtxt('../data/inputs/cambridge_work_commuter_lsoas_to_msoas/table_lsoas_to_msoas.txt')

In [ ]:
sample_index = 2143 
plt.figure(figsize=(10,10))
plt.scatter(x=np.mean(np.exp(tc_low_noise_log_destination_attractions),axis=0),y=destination_attraction,label='T.C. low noise',color='red')
plt.scatter(x=np.mean(np.exp(tc_high_noise_log_destination_attractions),axis=0),y=destination_attraction,label='T.C. high noise',color='green')
plt.scatter(x=np.mean(np.exp(tc_learned_noise_log_destination_attractions),axis=0),y=destination_attraction,label='T.C. learned noise',color='blue')
plt.scatter(x=np.mean(np.exp(pc_low_noise_log_destination_attractions),axis=0),y=destination_attraction,label='P.C. low noise',color='red', marker='x',s=200)
plt.scatter(x=np.mean(np.exp(pc_high_noise_log_destination_attractions),axis=0),y=destination_attraction,label='P.C. high noise',color='green', marker='x',s=200)
plt.scatter(x=np.mean(np.exp(pc_learned_noise_log_destination_attractions),axis=0),y=destination_attraction,label='P.C. learned noise',color='blue', marker='x',s=200)
# plt.scatter(x=np.exp(low_noise_log_destination_attractions)[sample_index],y=destination_attraction,label='low noise',color='red')
# plt.scatter(x=np.exp(high_noise_log_destination_attractions)[sample_index],y=destination_attraction,label='high noise',color='green')
# plt.scatter(x=np.exp(learned_noise_log_destination_attractions)[sample_index],y=destination_attraction,label='learned noise',color='blue')
plt.plot(np.arange(0,0.35,0.1),np.arange(0,0.35,0.1))
plt.legend()
plt.show()

In [ ]:
for prefix,name in {"tc":"TotalConstrained","pc":"ProductionConstrained"}.items():

    for noise_regime in ['low_noise','high_noise','learned_noise']:

        thetas = globals()[(f"{prefix}_{noise_regime}_thetas")]
        log_destination_attractions = globals()[(f"{prefix}_{noise_regime}_log_destination_attractions")]
        sim = getattr(SIM_Model,name)
        try:
            with ProgressBar(total=thetas.shape[0]) as progress_bar:
                temp = sim.log_flow_matrix_vectorised(
                    log_destination_attractions,
                    thetas,
                    origin_demand,
                    cost_matrix,
                    BAREBONE_CONFIG['table_total'],
                    progress_bar
                ) 
            exec(f"{prefix}_{noise_regime}_log_intensity = temp")
        except:
            raise ValueError(f"Log intensity function failed for {noise_regime} {name}")

In [ ]:
for prefix,name in {"tc":"TotalConstrained","pc":"ProductionConstrained"}.items():

    for noise_regime in ['low_noise','high_noise','learned_noise']:
        
        log_intensity = globals()[(f"{prefix}_{noise_regime}_log_intensity")]
        # mean_srmse = np.mean(SRMSE(tab=np.exp(log_intensity),tab0=table[np.newaxis,:].astype('int32')))
        # print(f"Mean SRMSE {name} {noise_regime} = {mean_srmse}")
        srmse_mean = SRMSE(tab=np.mean(np.exp(log_intensity),axis=0)[np.newaxis,:],tab0=table[np.newaxis,:].astype('int32'))
        print(f"SRMSE mean {name} {noise_regime} = {srmse_mean}")

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(30,15))

_vmin = 0.0
_vmax = 0.0025

im = ax[0].imshow(
    (np.abs(np.mean(np.exp(low_noise_log_intensity),axis=0)-table).T/np.sum(table)), 
    cmap=plt.cm.coolwarm, 
    interpolation='nearest',
    vmin=_vmin, 
    vmax=_vmax
)
ax[0].set_title('Low noise error',fontsize=16)
ax[0].set_ylabel('Destinations',fontsize=16)
ax[0].set_yticks(range(np.shape(low_noise_log_intensity[sample_index])[1]))
ax[0].set_xlabel('Origins',fontsize=16)
ax[0].set_xticks(range(np.shape(low_noise_log_intensity[sample_index])[0]))

im = ax[1].imshow(
    (np.abs(np.mean(np.exp(high_noise_log_intensity),axis=0)-table).T/np.sum(table)), 
    cmap=plt.cm.coolwarm, 
    interpolation='nearest', 
    vmin=_vmin, 
    vmax=_vmax
)
ax[1].set_title('High noise error',fontsize=16)
ax[1].set_ylabel('Destinations',fontsize=16)
ax[1].set_yticks(range(np.shape(table)[1]))
ax[1].set_xlabel('Origins',fontsize=16)
ax[1].set_xticks(range(np.shape(table)[0]))

im = ax[2].imshow(
    (np.abs(np.mean(np.exp(learned_noise_log_intensity),axis=0)-table).T/np.sum(table)), 
    cmap=plt.cm.coolwarm, 
    interpolation='nearest', 
    vmin=_vmin, 
    vmax=_vmax
)
ax[2].set_title('Learned noise error',fontsize=16)
ax[2].set_ylabel('Destinations',fontsize=16)
ax[2].set_yticks(range(np.shape(table)[1]))
ax[2].set_xlabel('Origins',fontsize=16)
ax[2].set_xticks(range(np.shape(table)[0]))


fig.colorbar(im, ax=ax ,fraction=0.046, pad=0.04)
plt.show()

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(30,15))

_vmax = max(
            np.max(np.mean(np.exp(low_noise_log_intensity),axis=0)),
            np.max(np.mean(np.exp(high_noise_log_intensity),axis=0)),
            np.max(np.mean(np.exp(learned_noise_log_intensity),axis=0))
        )
_vmin = min(
    np.min(np.mean(np.exp(low_noise_log_intensity),axis=0)),
    np.min(np.mean(np.exp(high_noise_log_intensity),axis=0)),
    np.min(np.mean(np.exp(learned_noise_log_intensity),axis=0))
)

im = ax[0].imshow(
    np.mean(np.exp(low_noise_log_intensity),axis=0).T, 
    cmap=plt.cm.coolwarm, 
    interpolation='nearest',
    vmin=_vmin, 
    vmax=_vmax
)
ax[0].set_title('Low Noise Intensity',fontsize=16)
ax[0].set_ylabel('Destinations',fontsize=16)
ax[0].set_yticks(range(np.shape(low_noise_log_intensity[sample_index])[1]))
ax[0].set_xlabel('Origins',fontsize=16)
ax[0].set_xticks(range(np.shape(low_noise_log_intensity[sample_index])[0]))

im = ax[1].imshow(
    np.mean(np.exp(high_noise_log_intensity),axis=0).T, 
    cmap=plt.cm.coolwarm, 
    interpolation='nearest',
    vmin=_vmin, 
    vmax=_vmax
)
ax[1].set_title('High Noise Intensity',fontsize=16)
ax[1].set_ylabel('Destinations',fontsize=16)
ax[1].set_yticks(range(np.shape(high_noise_log_intensity[sample_index])[1]))
ax[1].set_xlabel('Origins',fontsize=16)
ax[1].set_xticks(range(np.shape(high_noise_log_intensity[sample_index])[0]))

im = ax[2].imshow(
    np.mean(np.exp(learned_noise_log_intensity),axis=0).T, 
    cmap=plt.cm.coolwarm, 
    interpolation='nearest', 
    vmin=_vmin, 
    vmax=_vmax
)
ax[2].set_title('Learned Noise Intensity',fontsize=16)
ax[2].set_ylabel('Destinations',fontsize=16)
ax[2].set_yticks(range(np.shape(table)[1]))
ax[2].set_xlabel('Origins',fontsize=16)
ax[2].set_xticks(range(np.shape(table)[0]))

fig.colorbar(im, ax=ax ,fraction=0.046, pad=0.04)
plt.show()

# Export

In [ ]:
print('TotalConstrained')
print(np.isnan(tc_low_noise_signs).sum())
print(np.isnan(tc_low_noise_thetas).sum())
print(np.isnan(tc_low_noise_log_destination_attractions).sum())
print('\n')
print(np.isnan(tc_high_noise_signs).sum())
print(np.isnan(tc_high_noise_thetas).sum())
print(np.isnan(tc_high_noise_log_destination_attractions).sum())
print('\n')
print(np.isnan(tc_learned_noise_signs).sum())
print(np.isnan(tc_learned_noise_thetas).sum())
print(np.isnan(tc_learned_noise_log_destination_attractions).sum())
print('\n \n \n')
print('Production constrained')
print(np.isnan(pc_low_noise_signs).sum())
print(np.isnan(pc_low_noise_thetas).sum())
print(np.isnan(pc_low_noise_log_destination_attractions).sum())
print('\n')
print(np.isnan(pc_high_noise_signs).sum())
print(np.isnan(pc_high_noise_thetas).sum())
print(np.isnan(pc_high_noise_log_destination_attractions).sum())
print('\n')
print(np.isnan(pc_learned_noise_signs).sum())
print(np.isnan(pc_learned_noise_thetas).sum())
print(np.isnan(pc_learned_noise_log_destination_attractions).sum())

In [ ]:
for prefix,name in {"tc":"TotalConstrained","pc":"ProductionConstrained"}.items():

    for noise_regime in ['low_noise','high_noise','learned_noise']:

        try:
            thetas = globals()[(f"{prefix}_{noise_regime}_thetas")]
            log_destination_attractions = globals()[(f"{prefix}_{noise_regime}_log_destination_attractions")]
            signs = globals()[(f"{prefix}_{noise_regime}_signs")]
            current_metadata = globals()[(f"{noise_regime}_metadata")][name]
            export_competitive_method_outputs(
                current_metadata,
                signs,
                thetas,
                log_destination_attractions
            )
        except:
            raise Exception(f"Export failed for {name} {noise_regime}")
    
